In [2]:
import re
import pdb
import html
import json
import pandas as pd
from glob import glob
from fuzzywuzzy import fuzz

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
data_dir = r'D:/HCL ML Challenge/HCL ML Challenge Dataset/'
keys = ["Net current assets (liabilities)", "Creditors: amounts falling due within one year",
        "Debtors: amounts falling due within one year", "Total assets less current liabilities",
        "Total net assets (liabilities)", "Capital and reserves",
        "Cash at bank and in hand", "Issued share capital", "1 Ordinary Shares 1 of £ each",
        "0 Ordinary of £1 each", "Called up share capital not paid",
        "Prepayments and accrued income", "Creditors: amounts falling due after more than one year",
        "Provisions for liabilities", "Accruals and deferred income", "Authorised share capital",
        "Total Shareholder funds","Intangible assets",  "Tangible assets", 
        "Profit and loss account", "Reserves","Current Assets", "Net assets","Fixed Assets"]

In [4]:
# Function to remove the 
def rchop(line, value):
    if value and line.endswith(value):
        chop_line1 = line[:-len(value)].strip()
        if chop_line1.endswith(value):
            return chop_line1[:-len(value)].strip()
        return chop_line1
    return line

In [5]:
# Converting the unicode into html codes
def html_decode(string):
    modified_string = string.replace("\\u00c2","").replace("\\u0024","&#36").replace("\\u20AC","&#8364").replace(u"\\u00a3","&#163")
    return modified_string

In [6]:
# Function to extract the details for year 2019
def extractor(file_name, filepath):
    # dictionary where the lines from
    # text will be stored
    dict1 = {}
    i = 0

    with open(filepath) as fh:
        reset_values = True
        col = 0
        for line in fh:
            line = line.strip()
            line = re.sub(10 * ' ', ' ', line)
            k = 0
            
            # pdb.set_trace()
            # Check whether balance sheet contains details about 2019
            if (i < 3) and ("2019" in line):
                reset_values = False
                year_list = re.findall("\d+", line)
                year_list = [y for y in year_list if len(y) == 4]
                
                # Check the column for 2019
                if len(year_list) != 1:
                    col = year_list.index("2019")
                    print(f"{year_list[0]}, index of 2019 is {col}")

            key = ""
            
            for key_ in keys:
                
                # Regex to capture patterns like (23,000),23000,100.00, 23,000,00
                m = re.search(
                    "\((([0-9]*[.,])?[0-9]+)\)|\s{1,20}([0-9]*[.,])?[0-9]+\s{1,5}[^a-zA-Z]|\s{0,5}-|([0-9]*[.,"
                    "])?[0-9]+$",
                    line)
                if m:
                    value = m.group(0).strip().split()[0]
                else:
                    value = " "

                #                 print(f'line: {line},value:{value}')
                if key_ in line:
                    key = key_
                elif fuzz.partial_ratio(key_.lower(), line.lower()) > 80:
                    if value != " ":
                        if re.search("^\d+", line):
                            key = rchop(line, value)
                        else:
                            key = line.split(str(value))[0].strip().rstrip(str(value)).strip()
                    else:
                        key = line.strip()
                #                 else:
                #                     continue
                if i>2 and col != 0:
                    upd_values = re.findall("(\((([0-9]*[.,])?[0-9]+)\)$|-$|([0-9]*[.,])?[0-9]+$)", line)
#                     print(upd_values)
                    try:
                        value = upd_values[0][0]
                    except:
                        pass
                if key:
                    break
            if key:
                if ('(' in value) and (')' in value):
                    value = "-" + value.strip("()")
                elif value == " " or value == "":
                    value = "nan"
                #                 print(f"key:{key}, value:{value}")
                dict1[html.escape(key)] = html.escape(value.replace(",", "").replace(".00", "").replace("Â£", "&#163"))
            #               print(f"dict:{dict1}")
            i += 1
        #     pdb.set_trace()
        if reset_values:
            dict1 = dict1.fromkeys(dict1, "nan")

    #     out_file = open(data_dir + "jsons/X8XUG1JD.json", "w")
    #     json.dump(dict1, out_file, indent = 4, sort_keys = False)
    #     out_file.close()
    print(dict1)
    return json.dumps(dict1)

In [7]:
text_files = glob(data_dir+"*.txt")
files_data ={}
for text_file in text_files:
    file_name = text_file.split("\\")[-1].split(".")[0]
    print(f"Extracting from file: {file_name}")
    files_data[file_name] = html_decode(extractor(file_name, text_file))
    print(f"Extraction is sucessful from file: {file_name}")

df = pd.DataFrame(files_data.items())
df = df.rename(columns={0:'Filename',1:'Extracted Values'})
# df["Extracted Values"] = df["Extracted Values"].apply(lambda x: json.loads(str(x)))
# print(df.dtypes)

df.to_csv(data_dir+ "submission_file.csv",index=False)

Extracting from file: X8XUG1JD
2019, index of 2019 is 0
{'Current Assets': '312', 'Creditors: amounts falling due within one year': '-59', 'Net current assets (liabilities)': '253', 'Total assets less current liabilities': '253', 'Total net assets (liabilities)': '253', 'Capital and reserves': '253'}
Extraction is sucessful from file: X8XUG1JD
Extracting from file: X8XX000W
2019, index of 2019 is 0
{'Current assets': 'nan', 'Cash at bank and in hand': '1000', 'Net assets': '1000', 'Issued share capital': 'nan', '1000 Ordinary Shares 1 of Â£ each': '1000', 'Total Shareholder funds': '1000'}
Extraction is sucessful from file: X8XX000W
Extracting from file: X8XX00J6
{'Called up share capital not paid': '-', 'Fixed Assets': '516', 'Current Assets': '-', 'Prepayments and accrued income': '-', 'Creditors: amounts falling due within one year': '0', 'Net current assets (liabilities)': '0', 'Total assets less current liabilities': '516', 'Creditors: amounts falling due after more than one year'

{'Called up share capital not paid': '0', 'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Issued share capital': 'nan', '100 Ordinary Shares of Â£1.00 each': '100', 'Total Shareholder funds': '100'}
Extraction is sucessful from file: X8XZI954
Extracting from file: X8XZI9MH
2020, index of 2019 is 1
{'Called up share capital not paid': '-', 'Fixed Assets': '-', 'Current Assets': '500000', 'Prepayments and accrued income': '-', 'Net current assets (liabilities)': '500000', 'Total assets less current liabilities': '500000', 'Total net assets (liabilities)': '500000', 'Capital and reserves': '500000'}
Extraction is sucessful from file: X8XZI9MH
Extracting from file: X8XZI9Y0
{'Fixed assets': '578', 'Current assets': '112532', 'Creditors: amounts falling due within one year': '-21596', 'Net current assets': '90936', 'Total assets less current liabilities': '91514', 'Accruals and deferred income': '-3850', 'Net assets': '87604', 'Capital and reserves': '87664

{'Called up share capital not paid': '4', 'Current Assets': '7078', 'Creditors: amounts falling due within one year': '-7047', 'Net current assets (liabilities)': '31', 'Total assets less current liabilities': '35', 'Total net assets (liabilities)': '35', 'Capital and reserves': '35'}
Extraction is sucessful from file: X8XZIKDF
Extracting from file: X8XZIKJ6
2019, index of 2019 is 0
{'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Issued share capital': 'nan', '100 Ordinary Shares 1 of Â£ each': '100', 'Total Shareholder funds': '100'}
Extraction is sucessful from file: X8XZIKJ6
Extracting from file: X8XZIKOX
2019, index of 2019 is 0
{'Fixed Assets': '8460', 'Current Assets': '22756', 'Creditors: amounts falling due within one year': '-27702', 'Net current assets (liabilities)': '-4946', 'Total assets less current liabilities': '3514', 'Total net assets (liabilities)': '3514', 'Capital and reserves': '3514'}
Extraction is sucessful from file: X8XZIKOX


{'Called up share capital not paid': '2', 'Net assets': '2', 'Tssued share capital': 'nan', '2 Ordinary Shares 1 of Â£ each': '2', 'Total Shareholder funds': '2'}
Extraction is sucessful from file: X8XZIS17
Extracting from file: X8XZIS4B
{'Fixed Assets': '960', 'Current Assets': '1541', 'Creditors: amounts falling due within one year': '-1209', 'Net current assets (liabilities)': '332', 'Total assets less current liabilities': '1292', 'Total net assets (liabilities)': '1292', 'Capital and reserves': '1292'}
Extraction is sucessful from file: X8XZIS4B
Extracting from file: X8XZIS7K
{'Current assets': 'nan', 'Creditors: amounts falling due within one year': 'nan', 'Net current assets': 'nan', 'Total assets less current liabilities': 'nan', 'Accruals and deferred income': 'nan', 'Net assets': 'nan', 'Capital and reserves': 'nan', 'companies subject to the small companies regime. The profit and loss account           has not been delivered to the Registrar of Companies': 'nan'}
Extraction 

{'Called up share capital not paid': '1', 'Net assets': '1', 'Tssued share capital': 'nan', '1 Ordinary Share 1 of Â£ each': '1', 'Total Shareholder funds': '1'}
Extraction is sucessful from file: X8Y28E20
Extracting from file: X8Y28E28
{'Called up share capital not paid': '500', 'Net assets': '500', 'Tssued share capital': 'nan', '500 Ordinary Shares 1 of Â£ each': '500', 'Total Shareholder funds': '500'}
Extraction is sucessful from file: X8Y28E28
Extracting from file: X8Y28E4O
{'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Issued share capital': 'nan', '100 Ordinary Shares 1 of Â£ each': '100', 'Total Shareholder funds': '100'}
Extraction is sucessful from file: X8Y28E4O
Extracting from file: X8Y28E9V
2019, index of 2019 is 0
{'Net assets': '0', 'Reserves': '0'}
Extraction is sucessful from file: X8Y28E9V
Extracting from file: X8Y28EQP
2020, index of 2019 is 1
{'Net assets': '0', 'Reserves': '0'}
Extraction is sucessful from file: X8Y28EQP
Extract

...

{'Called up share capital not paid': '0', 'Current assets': 'nan', 'Cash at bank and in hand': '100.20', 'Net assets': '100.20', 'Issued share capital': 'nan', '2  B Ordinary Shares of Â£0.10 each': '0.20', '100 Ordinary Shares of Â£1.00 each': '100', 'Total Shareholder funds': '100.20'}
Extraction is sucessful from file: X8Y4U974
Extracting from file: X8Y4U9G1
{'Fixed assets': '262', 'Current assets': '33642', 'Creditors: amounts falling due within': 'nan', 'Net current assets': '24269', 'Total assets less current liabilities': '24531', 'Net assets': '24531', 'Capital and reserves': '24531', 'subject to the small companies regime. The profit and loss account           has not been delivered to the': 'nan'}
Extraction is sucessful from file: X8Y4U9G1
Extracting from file: X8Y4UBJF
2020, index of 2019 is 1
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', '1 ORDINARY Share of Â£ | each': '1', 'Total Shareholder funds': '1'}
Extr

{'Called up share capital not paid': '10', 'Net assets': '10', 'Tssued share capital': 'nan', '10 Ordinary Shares of Â£ | each': '10', 'Total Shareholder funds': '10'}
Extraction is sucessful from file: X8Y4UL6O
Extracting from file: X8Y4ULBL
2019, index of 2019 is 0
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', '1 Ordinary Share 1 of Â£ each': '1', 'Total Shareholder funds': '1'}
Extraction is sucessful from file: X8Y4ULBL
Extracting from file: X8Y4ULYO
2019, index of 2019 is 0
{'Net assets': '0', 'Reserves': '0'}
Extraction is sucessful from file: X8Y4ULYO
Extracting from file: X8Y4UMQ0
2019, index of 2019 is 0
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', '1 Ordinary Share 1 of Â£ each': '1', 'Total Shareholder funds': '1'}
Extraction is sucessful from file: X8Y4UMQ0
Extracting from file: X8Y4UN56
2019, index of 2019 is 0
{'Fixed assets': '23938', 'Current as

{'Current Assets': '23186', 'Creditors: amounts falling due within one year': '-7558', 'Net current assets (liabilities)': '15628', 'Total assets less current liabilities': '15628', 'Total net assets (liabilities)': '15628', 'Capital and reserves': '15628'}
Extraction is sucessful from file: X8Y4V20P
Extracting from file: X8Y4V357
2019, index of 2019 is 0
{'Called up share capital not paid': '-', 'Current Assets': '19894', 'Creditors: amounts falling due within one year': '-9019', 'Net current assets (liabilities)': '10875', 'Total assets less current liabilities': '10875', 'Total net assets (liabilities)': '10875', 'Capital and reserves': '10875'}
Extraction is sucessful from file: X8Y4V357
Extracting from file: X8Y4V3VF
2019, index of 2019 is 0
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', '1 Ordinary Share 1 of Â£ each': '1', 'Total Shareholder funds': '1'}
Extraction is sucessful from file: X8Y4V3VF
Extracting from file

{'Current assets': '2070', 'Creditors: amounts falling due within': 'nan', 'Net current assets': '1991', 'Total assets less current liabilities': '1991', 'Net assets': '1991', 'Capital and reserves': '1991', 'subject to the small companies regime. The profit and loss account          has not been delivered to the': 'nan'}
Extraction is sucessful from file: X8Y4VLLC
Extracting from file: X8Y4VMKJ
2019, index of 2019 is 0
{'Fixed Assets': '-', 'Current Assets': '-', 'Creditors: amounts falling due within one year': '-6000', 'Net current assets (liabilities)': '-6000', 'Total assets less current liabilities': '-6000', 'Total net assets (liabilities)': '-6000', 'Capital and reserves': '-6000'}
Extraction is sucessful from file: X8Y4VMKJ
Extracting from file: X8Y4VNMA
2019, index of 2019 is 0
{'Fixed Assets': '8792', 'Current Assets': '24563', 'Creditors: amounts falling due within one year': '-31912', 'Net current assets (liabilities)': '-7349', 'Total assets less current liabilities': '14

{'Current Assets': '4387', 'Creditors: amounts falling due within one year': '-350', 'Net current assets (liabilities)': '4037', 'Total assets less current liabilities': '4037', 'Creditors: amounts falling due after more than one year': '-35600', 'Total net assets (liabilities)': '-31563', 'Capital and reserves': '-31563'}
Extraction is sucessful from file: X8Y4WAVC
Extracting from file: X8Y4WB00
2019, index of 2019 is 0
{'Net assets': '0', 'Reserves': '0'}
Extraction is sucessful from file: X8Y4WB00
Extracting from file: X8Y4WDTE
2019, index of 2019 is 0
{'Called up share capital not paid': '2', 'Net assets': '2', 'Tssued share capital': 'nan', '2 Ordinary Shares 1 of Â£ each': '2', 'Total Shareholder funds': '2'}
Extraction is sucessful from file: X8Y4WDTE
Extracting from file: X8Y4WDW9
2019, index of 2019 is 0
{'Fixed Assets': '-', 'Current Assets': '100', 'Creditors: amounts falling due within one year': '0', 'Net current assets (liabilities)': '100', 'Total assets less current lia